VacationPy

Note
Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

Store Part I results into DataFrame

Load the csv exported in Part I to a DataFrame

In [2]:
file_path = "../output_data/city_weather_data.csv"
csv_df = pd.read_csv(file_path)
csv_df

City      Lat       Lng  Max Temp  Humidity  Cloudiness Country  \
0     east london -33.0153   27.9116     68.22      68.0       100.0      ZA   
1    punta arenas -53.1500  -70.9167     46.40      71.0        20.0      CL   
2        hermanus -34.4187   19.2345     64.99      76.0        14.0      ZA   
3     new norfolk -42.7826  147.0587     96.01      19.0        93.0      AU   
4        cidreira -30.1811  -50.2056     76.35      82.0        34.0      BR   
..            ...      ...       ...       ...       ...         ...     ...   
542        talara  -4.5772  -81.2719     73.65      81.0       100.0      PE   
543      impfondo   1.6381   18.0667     71.82      88.0       100.0      CG   
544        mudgee -32.5943  149.5871     88.00      32.0         0.0      AU   
545    santa cruz -17.8000  -63.1667     75.20      83.0        99.0      BO   
546      ilhabela -23.7781  -45.3581     75.22      82.0       100.0      BR   

     Wind Speed          Date  
0         11.32  1.611537e+09  
1         16.11  1.611537e+09  
2         10.00  1.611537e+09  
3          5.01  1.611537e+09  
4         15.23  1.611537e+09  
..          ...           ...  
542        8.66  1.611537e+09  
543        3.09  1.611537e+09  
544       12.66  1.611537e+09  
545        9.22  1.611537e+09  
546        4.27  1.611537e+09  

[547 rows x 9 columns]

Humidity Heatmap

Configure gmaps.

Use the Lat and Lng as locations and Humidity as the weight.

Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [5]:
coord = csv_df[['Lat', 'Lng']].astype(float)
csv_df = csv_df.dropna()

humidity = csv_df["Humidity"].astype(float)

fig = gmaps.figure(center=(30.0, 0.0), zoom_level=2, map_type="TERRAIN")
# fig = gmaps.figure(map_type='SATELLITE')

heat_layer = gmaps.heatmap_layer(coord, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3)

fig.add_layer(heat_layer)
# plt.savefig("../Images/HumidityHeatMap.png") #This is not saving, why? 
fig

Figure(layout=FigureLayout(height='420px'))

Create new DataFrame fitting weather criteria

Narrow down the cities to fit weather conditions.

Drop any rows will null values.

In [6]:
#Set up criteria for best weather conditions by filtering data from import csv and then continue
csv_df = csv_df.dropna()
# csv_df.head()
# csv_df['Max Temp'] = pd.to_numeric(csv_df['Max Temp'])

#Max temp >70, <80
filter_df = csv_df.loc[csv_df["Max Temp"] < 80, :]
filter_1 = filter_df.loc[filter_df["Max Temp"] > 70, :]

filter_1.head()
                                                     
#Winnd speed <10 mph
filter_2 = filter_1.loc[filter_1['Wind Speed'] < 10, :]

#Zero clouds ==
filter_3 = filter_2.loc[filter_2['Cloudiness'] == 0, :]

filter_3.head()

City      Lat      Lng  Max Temp  Humidity  Cloudiness Country  \
68         yuli   9.6989  10.2744     70.38      22.0         0.0      NG   
75   sao filipe  14.8961 -24.4956     72.21      68.0         0.0      CV   
103        chuy -33.6971 -53.4616     70.93      93.0         0.0      UY   
142  vila velha -20.3297 -40.2925     73.99      94.0         0.0      BR   
166   guarapari -20.6667 -40.4975     73.99      94.0         0.0      BR   

     Wind Speed          Date  
68         7.56  1.611537e+09  
75         5.53  1.611537e+09  
103        6.67  1.611537e+09  
142        3.44  1.611537e+09  
166        3.44  1.611537e+09

Hotel Map

Store into variable named hotel_df.

Add a "Hotel Name" column to the DataFrame.

Set parameters to search for hotels with 5000 meters.

Hit the Google Places API for each city's coordinates.

Store the first Hotel result into the DataFrame.

Plot markers on top of the heatmap.

In [7]:
#Store filtered df from above into a new variable named hotel_df
hotel_df = pd.DataFrame(filter_3)
# hotel_df.head()

#Add an empty column titled "Hotel Name"
hotel_df["Hotel Name"] = ""

hotel_df.head()

City      Lat      Lng  Max Temp  Humidity  Cloudiness Country  \
68         yuli   9.6989  10.2744     70.38      22.0         0.0      NG   
75   sao filipe  14.8961 -24.4956     72.21      68.0         0.0      CV   
103        chuy -33.6971 -53.4616     70.93      93.0         0.0      UY   
142  vila velha -20.3297 -40.2925     73.99      94.0         0.0      BR   
166   guarapari -20.6667 -40.4975     73.99      94.0         0.0      BR   

     Wind Speed          Date Hotel Name  
68         7.56  1.611537e+09             
75         5.53  1.611537e+09             
103        6.67  1.611537e+09             
142        3.44  1.611537e+09             
166        3.44  1.611537e+09

In [8]:
#Search for hotels with specified criteria by setting up a dictionary for params argument in requests.get() method

query_params = {
    "radius": 25000,
    "type": "lodging",
    "keyword": "hotel name",
    "key": g_key  
}

# Loop through the hote_df and run a lat/long search for each city
for index, row in hotel_df.iterrows():

    lat = row["Lat"]
    lng = row["Lng"]
    city = row["City"]
    country = row["Country"]
    
     # change location each iteration while leaving original params in place
    query_params['location'] = f"{lat},{lng}"
    
    #user the search term: "Hotel Name" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # make request
    response = requests.get(base_url, params=query_params)
    
    # print the hotelname url, avoid doing for public github repos in order to avoid exposing key
#     print(response)

#     # convert to json
    show_hotel = response.json()
#     pprint(show_hotel)
# Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = show_hotel["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result...skipping.")

Missing field/result...skipping.
Missing field/result...skipping.
Missing field/result...skipping.
Missing field/result...skipping.
Missing field/result...skipping.
Missing field/result...skipping.
Missing field/result...skipping.
Missing field/result...skipping.
Missing field/result...skipping.
Missing field/result...skipping.
Missing field/result...skipping.
Missing field/result...skipping.
Missing field/result...skipping.


In [9]:
#Replace any empty rows in "Hote Name" column with NAN
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)

In [10]:
#Drop values with NAN for final results
hotel_df = hotel_df.dropna()
hotel_df

City      Lat       Lng  Max Temp  Humidity  Cloudiness Country  \
142  vila velha -20.3297  -40.2925     73.99      94.0         0.0      BR   
205      collie -33.3667  116.1500     71.22      47.0         0.0      AU   
219    takoradi   4.8845   -1.7554     77.97      85.0         0.0      GH   
245       mecca  21.4267   39.8261     70.88      57.0         0.0      SA   
323  port blair  11.6667   92.7500     74.48      69.0         0.0      IN   

     Wind Speed          Date                       Hotel Name  
142        3.44  1.611537e+09           Sheraton Vitoria Hotel  
205        3.27  1.611537e+09  Bottlemart Express - Club Hotel  
219        6.78  1.611537e+09          Takoradi Standard Hotel  
245        3.36  1.611537e+09              Acoomadation Makkah  
323        9.93  1.611537e+09     Keys Select Hotel Aqua Green

In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [13]:
# Add marker layer ontop of heat map
# Display figure
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig2 = gmaps.figure(center=(0.0, 0.0), zoom_level=2)
fig.add_layer(marker_layer)
fig.add_layer(heat_layer)
#.savefig("../Images/HotelMap.png")
fig

Figure(layout=FigureLayout(height='420px'))